In [1]:
!pip install textacy==0.11.0

     |████████████████████████████████| 200 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.5 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 23.5 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 57.2 MB/s eta 0:00:01
     |████████████████████████████████| 645 kB 53.4 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 41.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 457 kB 30.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 123.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 14.4 MB/s eta 0:00:01


In [2]:
!pip3 install pandas==1.3.4

     |████████████████████████████████| 11.6 MB 2.3 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2


In [4]:
!pip install xlsxwriter

In [3]:
!python -m spacy download en_core_web_sm &>devnull

In [7]:
!pip install WordCloud

     |████████████████████████████████| 220 kB 2.4 MB/s eta 0:00:01
  Created wheel for WordCloud: filename=wordcloud-1.8.1-cp39-cp39-macosx_10_9_x86_64.whl size=154343 sha256=61e0eaeed78632f22789c23a0271800ebb01a8f054fd23258f62ff9f92b3a9b8
  Stored in directory: /Users/abhinavbajpai/Library/Caches/pip/wheels/f9/7a/dd/06ef8b5dfe5483f6204133c08eeb16c287cc2c05e290ae2fc0
Successfully built WordCloud


In [29]:
import spacy
import textacy
import os
import pandas as pd
import re
import html
import xlsxwriter
from collections import Counter
from matplotlib import pyplot as plt
import numpy as np

In [12]:
'Change path to data folder'
path = '../data/05202022_youtube_comments.pkl'
df = pd.read_pickle(path)
df = df.drop(0)
df.head()

,name,comment,published_at,likes,Reply Count,parent_id,is_parent
1,schmoyoho,Stream the track! https://open.spotify.com/alb...,2018-07-12T18:42:05Z,13950,188,UgywoWu8TaqkOzGi8HF4AaABAg,Yes
2,Roweman,I wonder if they caught that intruder,2022-04-23T00:46:20Z,1,0,UgyUDUe6xUHoX5ziaql4AaABAg,Yes
3,xxCordellMxx,Why did I say this out loud when I was younger?,2022-04-21T20:27:23Z,0,0,Ugx9p0We_6_2c5etHQZ4AaABAg,Yes
4,Nicholas DiGaetano,The rapist was never caught?,2022-04-21T04:17:40Z,0,0,Ugz-Y1LK6QpBOVv2WI54AaABAg,Yes
5,Tango Bodhi,This will never get old.,2022-04-20T18:27:49Z,0,0,UgyHtE_H7Y5OOda5gfZ4AaABAg,Yes


In [13]:
'Declaring Text Cleaning Functions'
RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')
def impurity(text, min_len=10):
    # returns the share of suspicious characters in a text
    if text == None or len(text) < min_len:
        return 0
    else:
        return len(RE_SUSPICIOUS.findall(text)) / len(text)

def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    # remove >>
    text = re.sub(r">\S+", ' ', text)
    # remove #
    text = re.sub(r"#\S+", ' ', text)
    # remove <<
    text = re.sub(r"<\S+", ' ', text)
    return text.strip()


In [14]:
'Additional Text cleaning Using Textacy'
import textacy.preprocessing as tprep
def normalize(text):
 'Removes additional text impurities using built-in textacy functions'
 text = tprep.normalize.hyphenated_words(text)
 text = tprep.normalize.quotation_marks(text)
 text = tprep.normalize.unicode(text)
 text = tprep.remove.accents(text)
 text = tprep.replace.urls(text, repl = '')
 text = tprep.replace.emails(text, repl = '') 
 text = tprep.replace.hashtags(text, repl = '') 
 text = tprep.replace.numbers(text, repl = '') 
 text = tprep.replace.phone_numbers(text, repl = '') 
 text = tprep.replace.user_handles(text, repl ='') 
 text = tprep.replace.emojis(text, repl = '')
 return text

In [15]:
# Applying clean and impurity function
df['comment_lower'] = df['comment'].str.lower()
df['clean_comments'] = df['comment_lower'].apply(lambda x: clean(str(x)))
df['clean_comments'] = df['clean_comments'].map(normalize)
df['impurity'] = df['clean_comments'].apply(impurity, min_len=20)
df[['clean_comments', 'impurity']].sort_values(by='impurity', ascending=False).head(20) 
df['year'] = df['published_at'].apply(lambda x : x[:4])
df['month'] = df['published_at'].apply(lambda x : x[5:7])
df = df.reset_index(drop=True)

# Lemmatization Using Spacy

In [16]:
nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [17]:
def extract_lemmas(doc, **kwargs):
    ''' Extract Lemmas '''
    extractedLemma = [t.lemma_ for t in textacy.extract.words(doc, **kwargs)]
    return extractedLemma

def extract_noun_phrases(doc, preceding_pos=['NOUN'], sep='_'):
    '''Extract Noun Phrases '''
    patterns = []
    for pos in preceding_pos:
        patterns.append(f"POS:{pos} POS:NOUN:+")
    spans = textacy.extract.matches.token_matches(doc, patterns=patterns)
    nounPhrase = [sep.join([t.lemma_ for t in s]) for s in spans]    
    return nounPhrase

def extract_nlp(doc):
    ''' Extract Various Combinations of POS'''
    posResults= {
        'lemmas': extract_lemmas(doc,
                                  exclude_pos=['PART', 'PUNCT',
                                             'DET', 'PRON', 'SYM', 'SPACE'],
                                filter_stops=True),
        'adjs_verbs': extract_lemmas(doc, include_pos=['ADJ', 'VERB']),
        'nouns': extract_lemmas(doc, include_pos=['NOUN', 'PROPN']),
        'noun_phrases': extract_noun_phrases(doc, ['NOUN']),
        'adj_noun_phrases': extract_noun_phrases(doc, ['ADJ']),
        'Verb_noun_phrases': extract_noun_phrases(doc, ['VERB'])}
    return posResults

nlp_columns = list(extract_nlp(nlp.make_doc('')).keys())

for col in nlp_columns:
    df[col] = None



In [18]:
batch_size = 100
for i in range(0, len(df), batch_size):
    docs = nlp.pipe(df['clean_comments'][i:i + batch_size])
    for j, doc in enumerate(docs):
        for col, values in extract_nlp(doc).items():
            df[col].iloc[i + j] = values

/Users/abhinavbajpai/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Creating N-grams

In [19]:
sentences = df.lemmas.values

In [21]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
bigram = Phrases(sentences, min_count=1, threshold=3, delimiter='_')
bigram_phraser = Phraser(bigram)

In [22]:
df['lemmasNgrams']=None
i=0
for sent in sentences:
  df.at[i, 'lemmasNgrams'] = bigram_phraser[sent]
  i=i+1

In [23]:
df[['lemmas','lemmasNgrams']].head(5)

,lemmas,lemmasNgrams
0,"[stream, track, proceed, dodson, gregorys, ant...","[stream, track, proceed_dodson, gregorys, anto..."
1,"[wonder, catch, intruder]","[wonder_catch, intruder]"
2,"[loud, young]","[loud, young]"
3,"[rapist, catch]","[rapist, catch]"
4,[old],[old]


In [24]:
df.to_pickle('../output/06062022_clean_data.pkl')

In [25]:
df.to_excel('../output/06062022_clean_data.xlsx', index= False, engine='xlsxwriter')

In [ ]:
# df = pd.read_pickle('../output/06062022_clean_data.pkl')

# Count words by Frequency

In [26]:
# Function to count words
def count_words(df, column= col, preprocess=None, min_freq=2):
  # process tokens and update counter
  def update(doc):
    tokens = doc if preprocess is None else preprocess(doc)
    counter.update(tokens)
  # create counter and run through all data
  counter = Counter()
  df[column].map(update)
  # transform counter into a DataFrame
  freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
  freq_df = freq_df.query('freq >= @min_freq')
  freq_df.index.name = 'token'
  return freq_df.sort_values('freq', ascending=False)

In [30]:
df_all_years = []
df['month'] = df['published_at'].apply(lambda x : x[5:7])
for year in df.year.unique():
  for month in df.month.unique():    
    df_year = df[(df['year']==year) & (df['month']==month)] 
    freq_df = count_words(df_year, column= 'lemmas')
    freq_df['year'] = year
    freq_df['month']= month
    freq_df['comment_count']= len(df_year)
    
    freq_df = freq_df.reset_index(drop=False)
    
    df_all_years.append(freq_df)


In [31]:
df_all_years = pd.concat(df_all_years)

In [32]:
df_all_years.head(-5)

,token,freq,year,month,comment_count
0,year,28,2018,07,327
1,song,22,2018,07,327
2,july,21,2018,07,327
3,watch,21,2018,07,327
4,lol,16,2018,07,327
...,...,...,...,...,...
4157,duck,2,2010,08,24433
4158,legal,2,2010,08,24433
4159,tweet,2,2010,08,24433
4160,dislick,2,2010,08,24433


In [33]:
df_all_years.to_csv(os.path.join('../output/', '06062022_words_by_years.csv')) # Output for Tableau Reports